In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/MANDATORY_CITATION.md
/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/MANDATORY_CITATION.md
/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient007/patient007_4d.nii
/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient007/Info.cfg
/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient007/patient007_frame01_gt.nii
/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient007/MANDATORY_CITATION.md
/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient007/patient007_frame07.nii
/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient007/patient007_frame01.nii
/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient007/patient007_frame07_gt.nii
/kaggle/input/automated-cardia

In [1]:
pip install keras==2.13.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.13.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import nibabel as nib
from skimage.transform import resize
from skimage.util import img_as_float,img_as_ubyte
from skimage import exposure
import glob
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import keras

scaler=MinMaxScaler()

In [ ]:
import random

In [ ]:
temp_img=nib.load('/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient001/patient001_frame01.nii').get_fdata()
mask=nib.load('/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient001/patient001_frame01_gt.nii').get_fdata()

plt.figure(figsize=(12,8))


plt.subplot(331)
plt.imshow(temp_img[:,:,n],cmap='gray')
plt.subplot(332)
plt.imshow(mask[:,:,n])

In [ ]:
import numpy as np
import nibabel as nib
import imageio
import matplotlib.pyplot as plt
from IPython.display import Image

# Load the NIfTI files
temp_img = nib.load('/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient001/patient001_frame01.nii').get_fdata()
mask = nib.load('/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/patient001/patient001_frame01_gt.nii').get_fdata()

# Create a list to store the images
images = []

# Create a figure and set its size
plt.figure(figsize=(12, 8))

# Loop through slices 0-15
for n in range(10):
    # Plot temp_img in subplot 1
    plt.subplot(331)
    plt.imshow(temp_img[:,:,n], cmap='gray')
    plt.title(f"temp_img Slice {n}")
    
    # Plot mask in subplot 2
    plt.subplot(332)
    plt.imshow(mask[:,:,n])
    plt.title(f"Mask Slice {n}")
    
    # Convert the current figure to an image
    plt.tight_layout()
    fig = plt.gcf()
    fig.canvas.draw()

    # Convert the figure to a numpy array
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    # Add the current image to the list
    images.append(image)
    
    # Clear the plot for the next iteration
    plt.clf()

# Create the GIF
gif_path = 'slices.gif'
imageio.mimsave(gif_path, images, fps=5)  # Adjust fps as needed

# Display the GIF
Image(filename=gif_path)


In [ ]:
temp_img.shape

In [ ]:
def rescale_img(temp_frame):
    img = nib.load(temp_frame)

    # Get the image data
    img_data = img.get_fdata()

    # Get the current pixel spacing
    current_spacing = img.header.get_zooms()

    # Calculate the scaling factors for resampling
    scaling_factors = [current_spacing[0] / 1.0, current_spacing[1] / 1.0, current_spacing[2]]

    # Resample the image data using interpolation
    resampled_data = resize(img_data, (int(img_data.shape[0] * scaling_factors[0]),
                                       int(img_data.shape[1] * scaling_factors[1]),
                                       int(img_data.shape[2] * scaling_factors[2])),
                            preserve_range=True)

    # Update the pixel spacing information
    new_spacing = (1.0, 1.0, current_spacing[2])  # Keep the original slice thickness
    img.header.set_zooms(new_spacing)

    
    return resampled_data


In [ ]:
def rescale_mask(temp_mask):
  
    img_mask = nib.load(temp_mask,mmap=False)

    # Get the image data
    img_data_mask = img_mask.get_fdata()
    img_data_mask=img_data_mask.astype(np.uint8)

    # Get the current pixel spacing
    current_spacing = img_mask.header.get_zooms()

    # Calculate the scaling factors for resampling
    scaling_factors = [current_spacing[0] / 1.0, current_spacing[1] / 1.0, current_spacing[2]]

    # Resample the image data using interpolation
    resampled_data_mask = resize(img_data_mask, (int(img_data_mask.shape[0] * scaling_factors[0]),
                                       int(img_data_mask.shape[1] * scaling_factors[1]),
                                       int(img_data_mask.shape[2] * scaling_factors[2])),
                            order=0,
                            preserve_range=True)

    # Update the pixel spacing information
    new_spacing = (1.0, 1.0, current_spacing[2])  # Keep the original slice thickness
    img_mask.header.set_zooms(new_spacing)
    
    return resampled_data_mask

In [ ]:
def crop_image(rescaled_img):
    # Get the shape of the resampled image
    resampled_shape = rescaled_img.shape

    # Calculate the starting and ending indices for cropping in each dimension
    start_x = (resampled_shape[0] - 224) // 2
    end_x = start_x + 224
    start_y = (resampled_shape[1] - 224) // 2
    end_y = start_y + 224

    # Perform center cropping
    frame_temp = rescaled_img[start_x:end_x, start_y:end_y, :16]
    
    return frame_temp

In [ ]:
def crop_mask(rescaled_mask):
    # Get the shape of the resampled image
    resampled_shape_mask = rescaled_mask.shape

    # Calculate the starting and ending indices for cropping in each dimension
    start_x = (resampled_shape_mask[0] - 224) // 2
    end_x = start_x + 224
    start_y = (resampled_shape_mask[1] - 224) // 2
    end_y = start_y + 224

    # Perform center cropping
    frame_gt_temp = rescaled_mask[start_x:end_x, start_y:end_y, :16]
    
    return frame_gt_temp

In [ ]:
def apply_clahe(image_slice):
    # Convert the image slice to float and rescale it to the range [0,1]
    image_slice = img_as_float(image_slice)
    image_slice = exposure.rescale_intensity(image_slice, out_range=(0, 1))
    
    # Apply CLAHE normalization with specified clip limit and tile grid size
    clahe_normalized_slice = exposure.equalize_adapthist(image_slice, clip_limit=0.03, nbins=256)
    
    # Convert the CLAHE-normalized slice to uint8 format
    return img_as_ubyte(clahe_normalized_slice)

In [ ]:
os.makedirs('/kaggle/working/ACDC_TrainingData/input_data_3channels/images/', exist_ok=True)
os.makedirs('/kaggle/working/ACDC_TrainingData/input_data_3channels/masks/', exist_ok=True)

In [ ]:
import shutil

# Specify the path to the directory you want to delete
directory_to_delete = '/kaggle/working/ACDC_TrainingData'

# Delete the directory and all its contents
shutil.rmtree(directory_to_delete)


In [ ]:
frame_list = sorted(glob.glob('/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/*/*frame*.nii'))
frame_list = [file for file in frame_list if not os.path.basename(file).endswith('_gt.nii')]
frame_list.sort(key=lambda x: os.path.basename(x))
mask_list = sorted(glob.glob('/kaggle/input/automated-cardiac-diagnosis-challenge-miccai17/database/training/*/*frame*_gt.nii'))
mask_list.sort(key=lambda x: os.path.basename(x))

for img, (frame_file, mask_file) in enumerate(zip(frame_list, mask_list), 1):   
    print("Now preparing image and masks number: ", img)
      
    temp_frame=frame_file
    rescaled_img=rescale_img(temp_frame)
    
    clahe_normalized_data = np.zeros_like(rescaled_img)
    # Iterate through each slice in the z-axis
    for i in range(rescaled_img.shape[2]):
        # Apply CLAHE to the current slice
        clahe_normalized_data[:, :, i] = apply_clahe(rescaled_img[:, :, i])
    
    temp_frame=crop_image(clahe_normalized_data)
    temp_frame=scaler.fit_transform(temp_frame.reshape(-1, temp_frame.shape[-1])).reshape(temp_frame.shape)
        
    temp_mask=mask_file
    
    rescaled_mask=rescale_mask(temp_mask)

    temp_mask=crop_image(rescaled_mask)
  
   
    temp_frame=np.stack([temp_frame]*3,axis=3)
    
    val, counts = np.unique(temp_mask, return_counts=True)
    
    if (1 - (counts[0]/counts.sum())) > 0.01:  #At least 1% useful volume with labels that are not 0
        print("Save Me")
        temp_mask= to_categorical(temp_mask, num_classes=4)
        
        np.save('/kaggle/working/ACDC_TrainingData/input_data_3channels/images/image_'+str(img)+'.npy', temp_frame)
        np.save('/kaggle/working/ACDC_TrainingData/input_data_3channels/masks/mask_'+str(img)+'.npy', temp_mask)
        
    else:
        print("I am useless")   

In [ ]:
img_frame=np.load('/kaggle/working/ACDC_TrainingData/input_data_3channels/images/image_2.npy')
mask_frame=np.load('/kaggle/working/ACDC_TrainingData/input_data_3channels/masks/mask_2.npy')

In [ ]:
mask_frame.shape

In [ ]:
import random

plt.figure(figsize=(12,8))

n=random.randint(0,mask_frame.shape[2])

plt.subplot(231)
plt.imshow(img_frame[:,:,n,2])
plt.subplot(232)
plt.imshow(mask_frame[:,:,n,3])


In [ ]:
pip install split-folders

In [ ]:
import splitfolders  # or import split_folders

input_folder = '/kaggle/working/ACDC_TrainingData/input_data_3channels/'
output_folder = '/kaggle/working/ACDC_TrainingData/input_data_224/'
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.8, .2), group_prefix=None) # default values

In [2]:
import numpy as np
import os

def load_img(img_dir, img_list):
    # Load images from a list of filenames
    images = []
    for image_name in img_list:
        if image_name.endswith('.npy'):
            image = np.load(os.path.join(img_dir, image_name))
            images.append(image)
#             print(f"Shape of {image_name}: {image.shape}")
    
           
    return np.array(images)

def imageLoader(img_dir, img_list, mask_dir, mask_list, batch_size):
    # Sort image and mask lists to ensure they correspond to each other
    img_list.sort()
    mask_list.sort()
    
    L = len(img_list)
    
    # Infinite loop to generate batches
    while True:
        batch_start = 0
        batch_end = batch_size
        
        while batch_start < L:
            # Determine the limit for the current batch
            limit = min(batch_end, L)
            
            # Load images and masks for the current batch
            X = load_img(img_dir, img_list[batch_start:limit])
            Y = load_img(mask_dir, mask_list[batch_start:limit])
            
            # Yield the images and masks as a tuple
            yield (X, Y)
            
            # Update batch start and end for the next iteration
            batch_start += batch_size
            batch_end += batch_size


In [ ]:
train_img_dir = "/kaggle/working/ACDC_TrainingData/input_data_224/train/images/"
train_mask_dir = "/kaggle/working/ACDC_TrainingData/input_data_224/train/masks/"
train_img_list=os.listdir(train_img_dir)
train_mask_list = os.listdir(train_mask_dir)

batch_size = 2

train_img_datagen = imageLoader(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list, batch_size)


img, msk = train_img_datagen.__next__()


In [ ]:
img_num = random.randint(0,img.shape[0]-1)
test_img=img[img_num]
test_mask=msk[img_num]
test_mask=np.argmax(test_mask, axis=3)

n_slice=random.randint(0, test_mask.shape[2])
plt.figure(figsize=(12, 8))

plt.subplot(221)
plt.imshow(test_img[:,:,n_slice, 0], cmap='gray')
plt.title('Image flair')
plt.subplot(222)
plt.imshow(test_img[:,:,n_slice, 1], cmap='gray')
plt.title('Image t1ce')
plt.subplot(223)
plt.imshow(test_img[:,:,n_slice, 2], cmap='gray')
plt.title('Image t2')
plt.subplot(224)
plt.imshow(test_mask[:,:,n_slice])
plt.title('Mask')
plt.show()

In [ ]:

from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.metrics import MeanIoU

kernel_initializer =  'he_uniform' #Try others if you want


################################################################
inputs = Input((224,224,16,3))
s = inputs

#Contraction path
c1 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(s)
c1 = Dropout(0.1)(c1)
c1 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
p1 = MaxPooling3D((2, 2, 2))(c1)

c2 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
c2 = Dropout(0.1)(c2)
c2 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
p2 = MaxPooling3D((2, 2, 2))(c2)

c3 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
c3 = Dropout(0.2)(c3)
c3 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
p3 = MaxPooling3D((2, 2, 2))(c3)

c4 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
c4 = Dropout(0.2)(c4)
c4 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
p4 = MaxPooling3D(pool_size=(2, 2, 2))(c4)

c5 = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
c5 = Dropout(0.3)(c5)
c5 = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)

#Expansive path 
u6 = Conv3DTranspose(128, (2, 2, 2), strides=(2, 2, 2), padding='same')(c5)
u6 = concatenate([u6, c4])
c6 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
c6 = Dropout(0.2)(c6)
c6 = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)

u7 = Conv3DTranspose(64, (2, 2, 2), strides=(2, 2, 2), padding='same')(c6)
u7 = concatenate([u7, c3])
c7 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
c7 = Dropout(0.2)(c7)
c7 = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)

u8 = Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(c7)
u8 = concatenate([u8, c2])
c8 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
c8 = Dropout(0.1)(c8)
c8 = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)

u9 = Conv3DTranspose(16, (2, 2, 2), strides=(2, 2, 2), padding='same')(c8)
u9 = concatenate([u9, c1])
c9 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
c9 = Dropout(0.1)(c9)
c9 = Conv3D(16, (3, 3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)

outputs = Conv3D(4, (1, 1, 1), activation='softmax')(c9)

model = Model(inputs=[inputs], outputs=[outputs])

model.summary()


In [ ]:
#Test if everything is working ok. 

print(model.input_shape)
print(model.output_shape)

In [8]:
train_img_dir = "/kaggle/working/ACDC_TrainingData/input_data_224/train/images/"
train_mask_dir = "/kaggle/working/ACDC_TrainingData/input_data_224/train/masks/"

val_img_dir = "/kaggle/working/ACDC_TrainingData/input_data_224/val/images/"
val_mask_dir = "/kaggle/working/ACDC_TrainingData/input_data_224/val/masks/"

train_img_list=os.listdir(train_img_dir)
train_mask_list = os.listdir(train_mask_dir)

val_img_list=os.listdir(val_img_dir)
val_mask_list = os.listdir(val_mask_dir)

In [ ]:
batch_size = 4

train_img_datagen = imageLoader(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list, batch_size)

val_img_datagen = imageLoader(val_img_dir, val_img_list, 
                                val_mask_dir, val_mask_list, batch_size)

#Verify generator.... In python 3 next() is renamed as __next__()
img, msk = train_img_datagen.__next__()

In [3]:
pip install segmentation-models-3D

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 3.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 65.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:

In [ ]:
import segmentation_models_3D as sm

In [ ]:
import pandas as pd
import numpy as np
import glob

# Define the columns
columns = ['0', '1', '2', '3']
df = pd.DataFrame(columns=columns)

# List of file paths for training masks
train_mask_list = sorted(glob.glob('/kaggle/working/ACDC_TrainingData/input_data_224/train/masks/*.npy'))

# Process each mask image
for img in range(len(train_mask_list)):
    temp_image = np.load(train_mask_list[img])
    temp_image = np.argmax(temp_image, axis=3)
    val, counts = np.unique(temp_image, return_counts=True)
    
    # Create a dictionary for counts
    conts_dict = dict(zip(columns, counts))
    
    # Convert the dictionary to a DataFrame and concatenate with the main DataFrame
    temp_df = pd.DataFrame([conts_dict], columns=columns)
    df = pd.concat([df, temp_df], ignore_index=True)

# Calculate the sum for each label
label_0 = df['0'].sum()
label_1 = df['1'].sum()
label_2 = df['2'].sum()
label_3 = df['3'].sum()

# Calculate total labels and number of classes
total_labels = label_0 + label_1 + label_2 + label_3
n_classes = 4

# Class weights calculation: n_samples / (n_classes * n_samples_for_class)
wt0 = round((total_labels / (n_classes * label_0)), 2)  # round to 2 decimals
wt1 = round((total_labels / (n_classes * label_1)), 2)
wt2 = round((total_labels / (n_classes * label_2)), 2)
wt3 = round((total_labels / (n_classes * label_3)), 2)

# Print the calculated weights
print(f'Weight for label 0: {wt0}')
print(f'Weight for label 1: {wt1}')
print(f'Weight for label 2: {wt2}')
print(f'Weight for label 3: {wt3}')


In [ ]:
wt0, wt1, wt2, wt3 = 0.28,5.69,7.36,7.49

dice_loss = sm.losses.DiceLoss(class_weights=np.array([wt0, wt1, wt2, wt3])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = ['accuracy', sm.metrics.IOUScore(threshold=0.5)]

#Fit the model 

steps_per_epoch = len(train_img_list)//batch_size
val_steps_per_epoch = len(val_img_list)//batch_size


model.compile(optimizer = "adam", loss=total_loss, metrics=metrics)


In [ ]:
history=model.fit(train_img_datagen,
          steps_per_epoch=steps_per_epoch,
          epochs=100,
          verbose=1,
          validation_data=val_img_datagen,
          validation_steps=val_steps_per_epoch,
          )

In [ ]:
model.save('/kaggle/working/ACDC_3d_take4.hdf5')

In [5]:
from keras.models import load_model

In [6]:
my_model = load_model('/kaggle/working/ACDC_3d_take4.hdf5',compile=False)

In [9]:
from keras.metrics import MeanIoU

batch_size=8 #Check IoU for a batch of images
test_img_datagen = imageLoader(val_img_dir, val_img_list, 
                                val_mask_dir, val_mask_list, batch_size)

#Verify generator.... In python 3 next() is renamed as __next__()
test_image_batch, test_mask_batch = test_img_datagen.__next__()

test_mask_batch_argmax = np.argmax(test_mask_batch, axis=4)
test_pred_batch = my_model.predict(test_image_batch)
test_pred_batch_argmax = np.argmax(test_pred_batch, axis=4)

n_classes = 4
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(test_pred_batch_argmax, test_mask_batch_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

1/1 [==============================] - 17s 17s/step
Mean IoU = 0.8136598


In [29]:
img_num = 5

test_img = np.load("/kaggle/working/ACDC_TrainingData/input_data_224/val/images/image_"+str(img_num)+".npy")

test_mask = np.load("/kaggle/working/ACDC_TrainingData/input_data_224/val/masks/mask_"+str(img_num)+".npy")
test_mask_argmax=np.argmax(test_mask, axis=3)

test_img_input = np.expand_dims(test_img, axis=0)
test_prediction = my_model.predict(test_img_input)
test_prediction_argmax=np.argmax(test_prediction, axis=4)[0,:,:,:]


1/1 [==============================] - 2s 2s/step


In [14]:
test_img_input.shape

(1, 224, 224, 16, 3)

In [ ]:
from matplotlib import pyplot as plt
import random

#n_slice=random.randint(0, test_prediction_argmax.shape[2])
n_slice = random.randint(0,15)
plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,n_slice,1], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(test_mask_argmax[:,:,n_slice])
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(test_prediction_argmax[:,:, n_slice])
plt.show()
print(n_slice)